Das Notebook ist dafür da die User Stories auszuführen. Bitte testet sie davor in eurem .main (und macht euer main in das gitignore). Hier also nur funktionierende User Stories!

Damit der import richtig funktioniert müsst ihr die init datei des jeweiligen Ordners (model/data_access/business_logic) pflegen.

In [2]:
import os
import shutil

original = "database/original.db"
working = "database/hotel_reservation_sample_working.db"
shutil.copyfile(original, working)



os.environ["DB_FILE"] = working


import business_logic.room_manager
import model
import data_access
import business_logic

Hotel_manager = business_logic.HotelManager()
Room_manager = business_logic.RoomManager()
Booking_manager= business_logic.BookingManager()

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit 
ich dasjenige auswählen kann, welches meinen Wünschen 
entspricht. Wünsche sind:

1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, 
damit ich das Hotel nach meinem bevorzugten Standort 
(Stadt) auswählen kann.

In [ ]:
by_city = Hotel_manager.get_hotels_by_city("Zürich")
print(by_city)

[Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
)]


1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [2]:
by_city_and_stars = Hotel_manager.get_hotels_by_city_and_stars("Zürich",6)
print(by_city_and_stars)

No hotels found in Zürich with 6 stars


1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).

In [3]:
by_max_guests = Hotel_manager.get_hotels_by_city_and_max_guests("Zürich", 3)
print(by_max_guests)

No hotels found in Zürich with a maximum of 3 guests


1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.

1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [4]:
show_user_friendly = Hotel_manager.read_hotel_userfriendly()
for hotels in show_user_friendly:
    print(hotels)

Name: Hotel Baur au Lac
Street: Bahnhofstrasse 1
City: Zürich
Stars: 5

Name: Four Seasons Hôtel des Bergues
Street: Rue du Rhône 42
City: Genève
Stars: 5

Name: Grand Hotel National
Street: Pilatusstrasse 15
City: Luzern
Stars: 5

Name: Bellevue Palace
Street: Marktgasse 59
City: Bern
Stars: 5

Name: Les Trois Rois
Street: Freiestrasse 10
City: Basel
Stars: 5



2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.

2.1. Ich möchte die folgenden Informationen pro Zimmer 
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung, 
Ausstattung, Preis pro Nacht und Gesamtpreis.

In [5]:
all = Room_manager.get_room_details_for_hotel(3)
print(all)


[Room(
  ID: 4
  Hotel ID: 3
  Room Number: 301
  Type ID: RoomType(
  ID: 4
  Beschreibung: Family Room
  Max Gäste: 5
  Facilities: None
)
  Price per night: 900.00
)]


2.2. Ich möchte nur die verfügbaren Zimmer sehen, sofern 
ich meinen Aufenthalt (von – bis) spezifiziert habe

3. Als Admin des Buchungssystems möchte ich die Möglichkeit haben, 
Hotelinformationen zu pflegen, um aktuelle Informationen im 
System zu haben.

3.1. Ich möchte neue Hotels zum System hinzufügen

In [6]:
#Hotel_manager.create_new_hotel("Test Hotel", 2, 3)
show_all = Hotel_manager.read_all_hotels()
print(show_all)

[Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
), Hotel(
  ID: 2
  Name: Four Seasons Hôtel des Bergues
  Stars: 5
  Address ID: Address(
  ID: 2
  Street: Rue du Rhône 42
  City: Genève
  Zipcode: 1204
)
  Rooms: []
), Hotel(
  ID: 3
  Name: Grand Hotel National
  Stars: 5
  Address ID: Address(
  ID: 3
  Street: Pilatusstrasse 15
  City: Luzern
  Zipcode: 6003
)
  Rooms: []
), Hotel(
  ID: 4
  Name: Bellevue Palace
  Stars: 5
  Address ID: Address(
  ID: 4
  Street: Marktgasse 59
  City: Bern
  Zipcode: 3011
)
  Rooms: []
), Hotel(
  ID: 5
  Name: Les Trois Rois
  Stars: 5
  Address ID: Address(
  ID: 5
  Street: Freiestrasse 10
  City: Basel
  Zipcode: 4051
)
  Rooms: []
)]


3.2. Ich möchte Hotels aus dem System entfernen


In [7]:
Hotel_manager.delete_hotel_by_id(9)
print(Hotel_manager.read_all_hotels())

[Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
), Hotel(
  ID: 2
  Name: Four Seasons Hôtel des Bergues
  Stars: 5
  Address ID: Address(
  ID: 2
  Street: Rue du Rhône 42
  City: Genève
  Zipcode: 1204
)
  Rooms: []
), Hotel(
  ID: 3
  Name: Grand Hotel National
  Stars: 5
  Address ID: Address(
  ID: 3
  Street: Pilatusstrasse 15
  City: Luzern
  Zipcode: 6003
)
  Rooms: []
), Hotel(
  ID: 4
  Name: Bellevue Palace
  Stars: 5
  Address ID: Address(
  ID: 4
  Street: Marktgasse 59
  City: Bern
  Zipcode: 3011
)
  Rooms: []
), Hotel(
  ID: 5
  Name: Les Trois Rois
  Stars: 5
  Address ID: Address(
  ID: 5
  Street: Freiestrasse 10
  City: Basel
  Zipcode: 4051
)
  Rooms: []
)]


3.3. Ich möchte die Informationen bestimmter Hotels 
aktualisieren, z. B. den Namen, die Sterne usw.

In [8]:
Hotel_manager.update_hotel(11, "Updated Hotel", 1, 2)
print(Hotel_manager.read_all_hotels())

[Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
), Hotel(
  ID: 2
  Name: Four Seasons Hôtel des Bergues
  Stars: 5
  Address ID: Address(
  ID: 2
  Street: Rue du Rhône 42
  City: Genève
  Zipcode: 1204
)
  Rooms: []
), Hotel(
  ID: 3
  Name: Grand Hotel National
  Stars: 5
  Address ID: Address(
  ID: 3
  Street: Pilatusstrasse 15
  City: Luzern
  Zipcode: 6003
)
  Rooms: []
), Hotel(
  ID: 4
  Name: Bellevue Palace
  Stars: 5
  Address ID: Address(
  ID: 4
  Street: Marktgasse 59
  City: Bern
  Zipcode: 3011
)
  Rooms: []
), Hotel(
  ID: 5
  Name: Les Trois Rois
  Stars: 5
  Address ID: Address(
  ID: 5
  Street: Freiestrasse 10
  City: Basel
  Zipcode: 4051
)
  Rooms: []
)]


4. Als Gast möchte ich ein Zimmer in einem bestimmten Hotel 
buchen, um meinen Urlaub zu planen.

In [ ]:
# hotel nach Name suchen
# create booking aufrufen

5. Als Gast möchte ich nach meinem Aufenthalt eine Rechnung 
erhalten, damit ich einen Zahlungsnachweis habe.
Hint: Fügt einen Eintrag in der «Invoice» Tabelle hinzu.

6. Als Gast möchte ich meine Buchung stornieren, damit ich nicht 
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice. 

7. Als Gast möchte ich eine dynamische Preisgestaltung auf der 
Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten 
Preis buchen kann.
Hint: Wendet in der Hochsaison höhere und in der Nebensaison 
niedrigere Tarife an.

8. Als Admin des Buchungssystems möchte ich alle Buchungen aller 
Hotels sehen können, um eine Übersicht zu erhalten.

In [3]:
t=Booking_manager.read_all_bookings()
print(t)

[Booking(
  ID: 1
  Room: Room(
  ID: 1
  Hotel ID: Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
)
  Room Number: 101
  Type ID: 1
  Price per night: 250.00
)
  Check In Date: 2025-06-01
  Check Out Date: 2025-06-05
  Total Amount of the Booking: 1000.0
  Guest:  
  ID: 1
  Guest's fullname: Hans Müller
  Guest's E-Mail: hans.mueller@example.ch
  Guest's Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
)
  Is cancelled: False
), Booking(
  ID: 2
  Room: Room(
  ID: 2
  Hotel ID: Hotel(
  ID: 1
  Name: Hotel Baur au Lac
  Stars: 5
  Address ID: Address(
  ID: 1
  Street: Bahnhofstrasse 1
  City: Zürich
  Zipcode: 8001
)
  Rooms: []
)
  Room Number: 102
  Type ID: 2
  Price per night: 400.00
)
  Check In Date: 2025-07-10
  Check Out Date: 2025-07-15
  Total Amount of the Booking: 2000.0
  Guest:  
  ID: 2
  Guest's fullname: Sophie Meier
  

9. Als Admin möchte ich eine Liste der Zimmer mit ihrer 
Ausstattung sehen, damit ich sie besser bewerben kann.


10.Als Admin möchte ich in der Lage sein, Stammdaten zu verwalten, 
z.B. Zimmertypen, Einrichtungen, und Preise in Echtzeit zu 
aktualisieren, damit das Backend-System aktuelle Informationen 
hat. Hint: Stammdaten sind alle Daten, die nicht von anderen Daten 
abhängen.